In [114]:
# -*- coding: utf-8 -*-

from clarifai.rest import ClarifaiApp
from clarifai.rest import Image as ClImage
import pandas as pd
import csv

## Configurando o Clarifai
Aqui carregamos o app do Clarifai com a chave da api e lemos o csv com as pinturas.

In [180]:
app = ClarifaiApp(api_key='670e2cfc02984504bde8044b30bc99d4')
paintings = pd.read_csv('../dados/paintings.csv') #read data and name columns

## Separando todos os conceitos
A partir do csv, tiremos extrair os principais conceitos das pinturas: Técnica, Autor e Escola.

In [181]:
all_concepts = []
todos_autores = []
todas_escolas = []
todas_tecnicas = []
i = 0
for index, row in paintings.iterrows():
    if i < 1000:
        tecnica = row['TECHNIQUE'].split(',')[0]
        autor = row['AUTHOR']
        escola = row['SCHOOL']
        all_concepts = all_concepts + [tecnica, autor, escola]
        todos_autores.append(autor)
        todas_escolas.append(escola)
        todas_tecnicas.append(tecnica)
        
        i = i+1
    
all_concepts = list(set(all_concepts))
todas_escolas = list(set(todas_escolas))
todos_autores = list(set(todos_autores))
todas_tecnicas = list(set(todas_tecnicas))

In [182]:
print(len(all_concepts), len(todas_escolas), len(todas_tecnicas),len(todos_autores))

216 19 76 121


## Criando as Imagens
Enviaremos as pinturas para o clarifai, cada imagem contém as informações dos conceitos presentes e dos conceitos não presentes (todos os conceitos possíveis - os conceitos presentes).

Rode o chunk abaixo se irá recriar imagens no app da Clarifai.

In [118]:
app.inputs.delete_all()

In [119]:
i = 0
for index, row in paintings.iterrows():
    if i <1001:
        print(i)
        tecnica = row['TECHNIQUE'].split(',')[0]
        autor = row['AUTHOR']
        url = row['URL']
        escola = row['SCHOOL']
        concepts_list = [tecnica, escola, autor]
        not_concepts_list = [i for i in all_concepts if i not in concepts_list]
        app.inputs.create_image_from_url(url, concepts=concepts_list, not_concepts=not_concepts_list)
        i = i +1
    else:
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

Run the following chunk if and only if there is a change in all_concepts

In [191]:
#app.models.delete('Paintings')
app.models.delete('Autor')

{'status': {'code': 10000, 'description': 'Ok'}}

## Criando e Carregando o Modelo

In [199]:
#model = app.models.create('Paintings', concepts=all_concepts)

model = app.models.get('Paintings')

In [198]:
#Decidir se vai usar varios mesmo ou só um.
modelo_escola = app.models.create('Escola', concepts=todas_escolas)
modelo_tecnica = app.models.create('Tecnica', concepts=todas_tecnicas)
modelo_autor = app.models.create('Autor', concepts=todos_autores)


ApiError: POST https://api.clarifai.com/v2/models FAILED(1533452676.5869486). status_code: 402, reason: Payment Required, error_code: 11006, error_description: Account limits exceeded, error_details: Request exceeds "custom concept" limit.
 >> Python client 2.2.3 with Python 3.6.4 on linux
 >> POST https://api.clarifai.com/v2/models
 >> REQUEST(1533452676.5869486) {
  "model": {
    "id": "Escola",
    "name": "Escola",
    "output_info": {
      "output_config": {
        "concepts_mutually_exclusive": false,
        "closed_environment": false
      },
      "data": {
        "concepts": [
          {
            "id": "German"
          },
          {
            "id": "Greek"
          },
          {
            "id": "Russian"
          },
          {
            "id": "English"
          },
          {
            "id": "Spanish"
          },
          {
            "id": "Scottish"
          },
          {
            "id": "Irish"
          },
          {
            "id": "Bel

## Treinando o Modelo

In [200]:
model = app.models.get('Paintings')
model.train()

## Testando os Resultados

In [137]:
def getInfo(prediction):
    output = prediction['outputs'][0]
    concepts = output['data']['concepts']

In [203]:
image = ClImage(url='https://www.wga.hu/detail/a/aachen/allegory.jpg')
image2 = ClImage(url='https://www.wga.hu/detail/a/ademollo/ark.jpg')
image3 = ClImage(url='https://www.wga.hu/detail/a/albani/1/4winter.jpg')
image4 = ClImage(url='https://upload.wikimedia.org/wikipedia/commons/f/f4/Aachen_Saint_Sebastian.jpg')
image5 = ClImage(url='https://www.wga.hu/detail/a/aelst/hunting2.jpg')
image6 = ClImage(url='https://www.wga.hu/detail/a/aertsen/christ.jpg')
model = app.models.get('Paintings')


In [207]:
columns = ['AUTHOR_EXPECTED', 'AUTHOR_PREDICTED', 'TECHNIQUE_EXPECTED', 'TECHNIQUE_PREDICTED','SCHOOL_EXPECTED', 'SCHOOL_PREDICTED', "MATCHES"]
df = pd.DataFrame(columns=columns)

i = 0
teste = paintings.head(1000).sample(10)
for index, row in teste.iterrows():
    tecnica = row['TECHNIQUE'].split(',')[0]
    autor = row['AUTHOR']
    url = row['URL']
    escola = row['SCHOOL']
    image = ClImage(url=url)
    result = model.predict([image])['outputs'][0]['data']['concepts']
    pred_autor = False
    pred_tecnica = False
    pred_escola = False
    total = 0
    for concept in result:
        conceito = concept['name']
        if ((not pred_autor) and conceito in todos_autores):
            pred_autor = conceito
            if pred_autor == autor:
                total += 1
        if ((not pred_escola) and conceito in todas_escolas):
            pred_escola = conceito
            if pred_escola == escola:
                total += 1
        if ((not pred_tecnica) and conceito in todas_tecnicas):
            pred_tecnica = conceito
            if pred_tecnica == tecnica:
                total +=1
    df_data = [autor, pred_autor, tecnica, pred_tecnica, escola, pred_escola]
    df_data.append(total)
    print(df_data)
    line = pd.DataFrame([df_data],columns=columns)    
    if i == 0:
        df = line
    else:
        df = df.append(line)
    i = i+1

['ANGELICO, Fra', 'ANGELICO, Fra', 'Tempera on wood', 'Tempera on wood', 'Italian', 'Italian', 3]
['ANDREA DEL SARTO', 'ANDREA DEL SARTO', 'Fresco', 'Fresco', 'Italian', 'Italian', 3]
['ANGELICO, Fra', 'ANGELICO, Fra', 'Tempera on wood', 'Fresco', 'Italian', 'Italian', 2]
['ARCIMBOLDO, Giuseppe', 'ARCIMBOLDO, Giuseppe', 'Oil on poplar wood', 'Oil on panel', 'Italian', 'Italian', 2]
['ANDREA DEL CASTAGNO', 'ANDREA DEL CASTAGNO', 'Fresco', 'Fresco', 'Italian', 'Italian', 3]
['ALTICHIERO da Zevio', 'ALTICHIERO da Zevio', 'Fresco', 'Fresco', 'Italian', 'Italian', 3]
['ARCIMBOLDO, Giuseppe', 'ARCIMBOLDO, Giuseppe', 'Oil on oak wood', 'Oil on panel', 'Italian', 'Italian', 2]
['AERTSEN, Pieter', 'AERTSEN, Pieter', 'Oil on wood', 'Oil on panel', 'Netherlandish', 'Netherlandish', 2]
['ALSLOOT, Denis van', 'ALSLOOT, Denis van', 'Oil on copper', 'Oil on canvas', 'Flemish', 'Flemish', 2]
['AST, Balthasar van der', 'AST, Balthasar van der', 'Oil on wood', 'Oil on panel', 'Dutch', 'Dutch', 2]


In [208]:
df.sample(10)

,AUTHOR_EXPECTED,AUTHOR_PREDICTED,TECHNIQUE_EXPECTED,TECHNIQUE_PREDICTED,SCHOOL_EXPECTED,SCHOOL_PREDICTED,MATCHES
0,"ANGELICO, Fra","ANGELICO, Fra",Tempera on wood,Fresco,Italian,Italian,2
0,"AST, Balthasar van der","AST, Balthasar van der",Oil on wood,Oil on panel,Dutch,Dutch,2
0,"AERTSEN, Pieter","AERTSEN, Pieter",Oil on wood,Oil on panel,Netherlandish,Netherlandish,2
0,ALTICHIERO da Zevio,ALTICHIERO da Zevio,Fresco,Fresco,Italian,Italian,3
0,ANDREA DEL SARTO,ANDREA DEL SARTO,Fresco,Fresco,Italian,Italian,3
0,"ALSLOOT, Denis van","ALSLOOT, Denis van",Oil on copper,Oil on canvas,Flemish,Flemish,2
0,"ANGELICO, Fra","ANGELICO, Fra",Tempera on wood,Tempera on wood,Italian,Italian,3
0,"ARCIMBOLDO, Giuseppe","ARCIMBOLDO, Giuseppe",Oil on poplar wood,Oil on panel,Italian,Italian,2
0,"ARCIMBOLDO, Giuseppe","ARCIMBOLDO, Giuseppe",Oil on oak wood,Oil on panel,Italian,Italian,2
0,ANDREA DEL CASTAGNO,ANDREA DEL CASTAGNO,Fresco,Fresco,Italian,Italian,3


In [ ]:
img1 = ClImage(url="https://samples.clarifai.com/puppy.jpeg", concepts=['boscoe'], not_concepts=['our_wedding'])
img2 = ClImage(url="https://samples.clarifai.com/wedding.jpg", concepts=['our_wedding'], not_concepts=['cat','boscoe'])

app.inputs.bulk_create_images([img1, img2])